# Simple hierarchical search

Requires following steps:

- chunk documents
- generate embeddings
- set up db table and sqlite-vss
- populate db table
- test queries
- simple UI

## Chunk documents
As I already have tools for working with Sphinx XML, and given that this work is partly related to developing Jupyter Book / Sphinx tools to work with OU-XML, let's start there...

First up, we need to be be able to chunk markdown documents. We can render Jupyter notebook `.ipnyb` documents, as well as MyST markdown, into Sphinx-XML by building using `jb build ./tests/book --all --builder custom --custom-builder xml` (I really need to do some digging to find the Python calls used by the command-line command, rather than having to use the CLI.)

Let's start with a hack to just convert a single file. Do this by creating a dummy `toc.yml` file and running the builder:

In [162]:
# %pip install lxml

import subprocess
from lxml import etree
from pathlib import Path

_ = subprocess.run(
    "jb build . --all --builder custom --custom-builder xml",
    shell=True,
    cwd=".",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

# We could perhaps get the name part of the filename
# by parsing the _toc.yml file.
xml_out = Path(".") / "_build" / "xml" / f"dummy.xml"


def create_top_section_root(xml_out):
    tree = etree.parse(
        xml_out, parser=etree.XMLParser(strip_cdata=False, remove_blank_text=True)
    )
    etree.cleanup_namespaces(tree)
    root = tree.getroot().find("section")
    # result = root.find(".//title").getnext()
    return tree, root


tree, root = create_top_section_root(xml_out)

xml_bytes = etree.tostring(root, pretty_print=True, encoding="utf-8")

print(xml_bytes.decode("utf-8"))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<section classes="tex2jax_ignore mathjax_ignore" docname="dummy" header_level="1" ids="lots-of-interesting-facts-about-snafu" names="lots\ of\ interesting\ facts\ about\ snafu">
  <title>Lots of Interesting Facts about Snafu</title>
  <paragraph>The capital of Baswas is foobar.</paragraph>
  <section docname="dummy" header_level="2" ids="a-history-of-junkwap" names="a\ history\ of\ junkwap">
    <title>A history of <emphasis>Junkwap</emphasis></title>
    <paragraph>Junkwap was founded in 1874. It is mainly green and blue.</paragraph>
    <paragraph>Here is a list:</paragraph>
    <bullet_list bullet="-">
      <list_item>
        <paragraph>one item</paragraph>
      </list_item>
      <list_item>
        <paragraph>another item</paragraph>
      </list_item>
    </bullet_list>
    <paragraph>The sun never wrinkles.</paragraph>
    <section docname="dummy" header_level="3" ids="the-asvrograde-of-junkwap" names="the\ asvrograde\ of\ junkwap">
      <title>The asvrograde of <emphasis>Ju

Inspection of the generated XML shows:

- a root `<document>` element with a `source` attribute giving the path to the source markdown file;
- at each markdown heading, a new `<section>` tag/block is introduced that includes the following attributes:
    - `header_level`: the header level is also the section level / depth;
    - `names`: an escaped, decased version of the section heading;
    - `ids`: a within document identifier generated from the section heading; this name not be unique across different documents; within a document, if a section heading is duplicatesd, the first `ids` is set in the normal way, and a new id created for duplicates; a `dupnames` attribute is also set for all duplicated sections in the `<section>` tag that contains the duplicated escaped `names` value;
- the first child of the section block is a `<title>` tag; this tag contains the heading text (that is, the section heading);
- implicitly, each section contains its subsections;
- sections contain `<paragraph>` elements and other block level chunks, such as lists. Within list elements, we also also have paragraphs, so paragtaphs may provide a useful level of chunking.

## Generating section chunks

The following chunking strategy recursively iterates through each section in a provided Sphinx-XML document.

Each chunk is flattened to simple, unformatted, text which can be used to generate embeddings or as part of an LLM prompt.

Paragraph level chunks are also extracted and flattened within each section.

In [160]:
from ou_xml_validator.utils import flatten_to_text

# Restate the root definition here so we can
# rerun this cell on a pristine root element
tree, root = create_top_section_root(xml_out)

sections = {}


def post_stop(text):
    """Add a trailing period/full stop if not exists."""
    return f"{text}." if not text.strip().endswith(".") else text.strip()

# Simple repair script for titles so they end with a .
# This is useful for sentence chunking
# There is an issue that if the content preceding the title
# has no ".", we don't get a sentence break.
title_tags = root.xpath("//title")
for title_tag in title_tags:
    if not title_tag.text.strip().endswith("."):
        period_node = etree.Element("text")
        period_node.text = "."
        title_tag.append(period_node)

def chunk_documents(root, sections=None):
    sections = {} if sections == None else sections
    # Find each section within the current section
    for section in root.findall("./section"):
        section_id = section.get("ids")
        sections[section_id] = {}
        section_ = sections[section_id]
        section_["title"] = flatten_to_text(section.find("./title")).strip()

        # Find paragraphs in the section and flatten each one
        # Paragraphs should not be in a child section
        # Relative path given by:
        # tree.getpath(p)[len(tree.getpath(section)) :]
        section_["paragraph_text"] = [
            post_stop(flatten_to_text(p))
            for p in section.findall(".//paragraph")
            if "section" not in tree.getpath(p)[len(tree.getpath(section)) :]
        ]

        # For lists, should we also
        # group all the items as a single paragraph?

        # Flatten text in section
        section_["full_flat_text"] = post_stop(flatten_to_text(section))

        section_["subsections"] = section.findall("./section")
        # Flatten text in a subsection
        section_["flat_subsections_text"] = [
            post_stop(flatten_to_text(s)) for s in section_["subsections"]
        ]

        # Recurse through subsections
        chunk_documents(section, sections)

        # Convert section refernce to id
        section_["subsections"] = [s.get("ids") for s in section_["subsections"]]

    return sections


sections = chunk_documents(root)

sections.keys()

['Lots of Interesting Facts about Snafu .', 'A history of  Junkwap .', 'The asvrograde of  Junkwap .', 'A part .', 'The oblograde of Junkway .', 'A part .', 'The Decline of Junkwap .']


dict_keys(['a-history-of-junkwap', 'the-asvrograde-of-junkwap', 'a-part', 'the-oblograde-of-junkway', 'id1', 'the-decline-of-junkwap'])

In [158]:
sections["a-history-of-junkwap"]

{'title': 'A history of  Junkwap .',
 'paragraph_text': ['Junkwap was founded in 1874. It is mainly green and blue.',
  'Here is a list:.',
  'one item.',
  'another item.',
  'The sun never wrinkles.'],
 'full_flat_text': ' A history of  Junkwap . Junkwap was founded in 1874. It is mainly green and blue. Here is a list:   one item  another item The sun never wrinkles.  The asvrograde of  Junkwap . The asvrograde of Junkwap is quite fruity and smells mainly of cheese.  A part . Some bits  The oblograde of Junkway . The oblograde of Junkwap is a round square whose in a world with a really strange geometry where the internal angles of the round square, which is a trianle, add up to 243 degrees.  A part . Some more bits.',
 'subsections': ['the-asvrograde-of-junkwap', 'the-oblograde-of-junkway'],
 'flat_subsections_text': [' The asvrograde of  Junkwap . The asvrograde of Junkwap is quite fruity and smells mainly of cheese.  A part . Some bits.',
  ' The oblograde of Junkway . The oblograd

Within each section, we can now generate embeddings on:

- the section title;
- the paragraph text that is not contained within a child section element; (this is available at paragraph level but could be joined);
- the full flat text of the whole section (including flattened child sections);
- the flat text of each child section.

We can generate embeddings using [`sentence-transformers`](https://www.sbert.net/docs/pretrained_models.html), although we are limited to sentences of about 300-400 words. For many paragraphs, this will be okay, but longer paragraphs will either need truncating or aggregating. For example, we could return an embedding for a long paragraph by aggregating over embeddings of particular chunks, either by creating a single aggregate embedding, such as a mean embedding, or in a search strategy that matches using the best match over all partial paragraph embeddings associated with a paragraph.

For long paragraphs, we could use something liuke the `spacy` sentence chunker to chunk our sentences and then build up chinks that as are long as possible.

In [161]:
# %pip install spacy
# Install simple model for sentence splitting
# import spacy.cli
# spacy.cli.download("en_core_web_sm")

import spacy
import math

nlp = spacy.load("en_core_web_sm")

text = sections["a-history-of-junkwap"]["full_flat_text"]

doc = nlp(text)


def paragraph_chunks(text, maxlen=350):
    """Generate chunks around paragraphs and sentences."""
    doc = nlp(text)
    sentences = list(doc.sents)
    chunk_start_sent = sentences[0]
    chunks = [chunk_start_sent]
    for sent in sentences[1:]:
        chunk_len = len(chunks[-1])
        next_sent_len = len(sent)
        proposed_len = chunk_len + next_sent_len
        if proposed_len < maxlen:
            chunks[-1] = doc[chunk_start_sent.start : sent.end]
        else:
            chunks.append(sent)
            chunk_start_sent = sent

    # Cap the length of chunks, if necesary by splitting paragraphs
    newchunks = []
    for chunk in chunks:
        chunk_len = len(chunk)
        if chunk_len > maxlen:
            # If a chunk is overlong, split it into N equal size partial chunks,
            # for smallest N where partial chunk size < maxlen
            chunk_block = int( chunk_len / math.ceil(chunk_len / maxlen))
            for i in range(0, len(chunk), chunk_block):
                newchunks.append(chunk[i : i + chunk_block])
        else:
            newchunks.append(chunk)
    #chunks = [c.text for c in newchunks]
    return chunks, newchunks


paragraph_chunks(text, 10)

([ A history of  Junkwap .,
  Junkwap was founded in 1874.,
  It is mainly green and blue.,
  Here is a list:   one item  another item The sun never wrinkles.  ,
  The asvrograde of  Junkwap .,
  The asvrograde of Junkwap is quite fruity and smells mainly of cheese.  ,
  A part .,
  Some bits  The oblograde of Junkway .,
  The oblograde of Junkwap is a round square whose in a world with a really strange geometry where the internal angles of the round square, which is a trianle, add up to 243 degrees.  ,
  A part . Some more bits.],
 [ A history of  Junkwap .,
  Junkwap was founded in 1874.,
  It is mainly green and blue.,
  Here is a list:   one item,
   another item The sun never wrinkles.,
   ,
  The asvrograde of  Junkwap .,
  The asvrograde of Junkwap is quite fruity,
  and smells mainly of cheese.  ,
  A part .,
  Some bits  The oblograde of Junkway .,
  The oblograde of Junkwap is a round square whose,
  in a world with a really strange geometry where,
  the internal angles of th

## Generating embeddings

One of the simplest way of generating embeddings is to use the Python [`sentence-transformers` package](https://www.sbert.net/).


In [86]:
# %pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util

# The model will be automatically downloaded if required
SENTENCE_TRANSFORMER_MODEL = "multi-qa-MiniLM-L6-cos-v1"
model = SentenceTransformer(SENTENCE_TRANSFORMER_MODEL)

# The default cut off is length 128 tokens
MAX_SENTENCE_TRANSFORMER_TOKENS = 500

# Max length is 510, but there be overhead...
MAX_SENTENCE_TRANSFORMER_TOKENS = min(MAX_SENTENCE_TRANSFORMER_TOKENS, 300)
model.max_seq_length = MAX_SENTENCE_TRANSFORMER_TOKENS

_query = "How big is London"
_responses = [
    "London has 9,787,426 inhabitants at the 2011 census",
    "London is known for its finacial district",
    "London is known for its finacial district",
]

# Generate embdding for one record
query_embedding = model.encode(_query)
# Generate emmbddings for mutliple records
passage_embedding = model.encode(_responses)

#Find match score between query embedding and each response embedding
print("Similarity:", util.dot_score(query_embedding, passage_embedding))


Similarity: tensor([[0.5472, 0.6330, 0.6330]])


The embedding vector returned by the sentence tranformer is a `numpy.ndarray`. Mean aggregate embeddings can be generated by using `np.mean(INDIVIDUAL_EMBEDDINGS, axis=0)`.

We can create ene

## Vector search

We can search across the generated embeddings using a SQLite3 database using the [`sqlite-vss`](https://github.com/asg017/sqlite-vss) extension installed [[about](https://observablehq.com/@asg017/introducing-sqlite-vss)].

In [87]:
# Install sqlite_utils, sqlite-vss 
#%pip install --upgrade sqlite_utils sqlite-vss sqlite-utils-sqlite-vss

# MacOS extension installation  :
# https://til.simonwillison.net/sqlite/sqlite-extensions-python-macos

# Check install
import sqlite3
import sqlite_vss

# Manually enable extension
db_ = sqlite3.connect(":memory:")
db_.enable_load_extension(True)
sqlite_vss.load(db_)

(version,) = db_.execute("select vss_version()").fetchone()
version

NameError: name 'db' is not defined

We can also create a `sqlite_utils.Database` with the `sqlite_vss` extension loaded:

In [ ]:
from sqlite_utils import Database

# Create the database
# db = Database(memory=True)
db = Database("hsearch_demo.db", recreate=True)

# Enable the extension
db.conn.enable_load_extension(True)
sqlite_vss.load(db.conn)

db

<Database <sqlite3.Connection object at 0x106de2890>>

One possible structure for an appropriate database table is as follows:

- `full_text`: the markdown text of the whole chunk;
- `raw_embedding`: an embedding for the whole chunk;
- `agg_embedding`: an aggregate embedding that is also a function of the raw embedding of child chunks;
- `doc_id`: for example, based on the document path;
- `section_id`: this could be the section identifier; it will only be guaranteed unique across documents if combined with the document path or identifier, for example;
- `parent_section_id`: the identifier of the parent section;
- `child_section_ids`: a list of child section identifiers; to reconstruct full text from section parts, we would need to have "top level" text and the section references in the correct locations within that text;
- `heading`: the section heading;
- `heading_embedding`: an embedding generated from the heading text;
- `section_level`: for example, the `header_level` in the Sphinx generated XML.

The `sqlite_utils.Database` is easier to work with. For example, we can triviually create a table on the database: